In [2]:
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
from ase.neighborlist import neighbor_list as make_neighbor_list
import numpy as np
from ase import Atoms
from tqdm import trange, tqdm
from pprint import pprint

In [2]:
def smooth_cut_s_function(
    r_ij_norm:torch.tensor,
    r_cutoff:float
)->torch.tensor:
    """DeepPot-SE"""
    r_cutoff_smth=0.1
    s_vec=torch.zeros_like(r_ij_norm)
    flag=(r_ij_norm<=r_cutoff)

    u=(r_ij_norm[flag]-r_cutoff_smth)/(r_cutoff-r_cutoff_smth)
    s_vec[flag]=(u*u*u(-6*u*u+15*u-10)+1)/r_ij_norm[flag]

    return s_vec

In [3]:
class ThreeLayerPerceptron(nn.Module):
    def __init__(self,input_size,hidden_size,output_size,activation_function):
        super(ThreeLayerPerceptron,self).__init__()
        self.fc1=nn.Linear(input_size,hidden_size)
        self.fc2=nn.Linear(hidden_size,hidden_size)
        self.fc3=nn.Linear(hidden_size,output_size)
        self.activation_function=activation_function
    
    def forward(self,x):
        x=self.fc1(x)
        x=self.activation_function(x)
        x=self.fc2(x)
        x=self.activation_function(x)
        x=self.fc3(x)
        return x

In [4]:
def coords_to_relative_coords(
    self,
    coords:torch.tensor,
    atom_i_idxs:torch.tensor,
    atom_j_idxs:torch.tensor,
    shift:torch.tensor
):
    coords_concat=torch.concat((
        coords,
        torch.full((1,3),1e5)
    ),dim=0)
    relative_coords=coords_concat[atom_j_idxs]-coords_concat[atom_i_idxs]+shift

    return relative_coords

In [13]:
def smooth_cut_s_function(
    r_ij_norm:torch.tensor, 
    r_cutoff:float, 
)->torch.tensor:
    """DeepPot-SE function s(r_ij) of tensor version
    As r_ij becomes larger than r_cutoff_smth, the output of the function begins to decrease, 
    and as more r_ij becomes larger, the output of the function smoothly becomes zero.
    Parameters
    ----------
        r_ij_norm : torch.tensor  
            shape : (Coordination Num of atom i,)
        r_cutoff : float
            cutoff
    Returns
    -------
        s_vec : torch.tensor
            shape : (Coordination Num of atom i,)
    """
    r_cutoff_smth = 0.1

    s_vec = torch.zeros_like(r_ij_norm)
    
    flag = (r_ij_norm <= r_cutoff)

    u = (r_ij_norm[flag] - r_cutoff_smth) / (r_cutoff - r_cutoff_smth)
    s_vec[flag] =  (u*u*u*(-6*u*u + 15*u - 10) + 1) / (r_ij_norm[flag])

    return s_vec

In [6]:
# Natoms=4
# coords=torch.arange(Natoms*3).reshape(Natoms,3)
coords=torch.tensor([[1.0,2.0,3.0],[4.0,5.0,6.0]],dtype=torch.float)
pprint(coords)

tensor([[1., 2., 3.],
        [4., 5., 6.]])


In [7]:
coords_concat=torch.concat((coords,torch.full((1,3),1e5)),dim=0)
coords_concat

tensor([[1.0000e+00, 2.0000e+00, 3.0000e+00],
        [4.0000e+00, 5.0000e+00, 6.0000e+00],
        [1.0000e+05, 1.0000e+05, 1.0000e+05]])

In [8]:
Natoms=2
sel=1
shift=10.0*torch.ones(Natoms*sel*3).reshape(Natoms*sel,3)
relative_coords=coords_concat[1]-coords_concat[0]+shift
relative_coords

tensor([[13., 13., 13.],
        [13., 13., 13.]])

In [29]:
relative_coords=torch.tensor([[1.0,2.0,3.0],[4.0,5.0,6.0]])
generalized_coords=torch.full((relative_coords.shape[0],4),1e+2)
r_ij_norm=torch.linalg.norm(relative_coords,dim=1)
# r_cutoff=6.0
s_vec=smooth_cut_s_function(r_ij_norm,r_cutoff)
generalized_coords[:,0]=s_vec
generalized_coords[:,1:]=relative_coords
generalized_coords[:,1:]=generalized_coords[:,1:]*s_vec
# print(relative_coords*s_vec.view(-1,1))
generalized_coords

tensor([0.0770, 0.0000])


RuntimeError: The size of tensor a (3) must match the size of tensor b (2) at non-singleton dimension 1

In [40]:
a = torch.ones((2, 3))
b = a.view(-1, 2) # 最後のdimのsizeを2に, 他のdimは"潰す"
print("# view(-1,2)で最後のdimのsizeを2に, 他のdimは潰す:")
print(b)

# view(-1,2)で最後のdimのsizeを2に, 他のdimは潰す:
tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])


In [ ]:
class DeepPotSEModel(nn.Module):
    def __init__(self,
        max_atom_type:int,
        atom_type_embed_nchanl:int,
        m1:int,
        m2:int,
        fitting_hidden_size:int,
        r_cutoff:float,
        activation_function,
        sel:int
    ):
        super(DeepPotSEModel,self).__init__()

        self.r_cutoff=r_cutoff
        self.max_atom_type=max_atom_type
        self.m1=m1
        self.m2=m2
        self.activation_function=activation_function
        self.sel=sel
        self.atom_type_embed_nchanl=atom_type_embed_nchanl

        self.atom_type_embed_net=ThreeLayerPerceptron(
            input_size=max_atom_type+1,
            hidden_size=atom_type_embed_nchanl,
            output_size=atom_type_embed_nchanl,
            activation_function=activation_function
        )

        self.embed_net=ThreeLayerPerceptron(
            input_size=1+2*atom_type_embed_nchanl,
            hidden_size=m1,
            output_size=m1,
            activation_function=activation_function
        )

        self.fitting_net=ThreeLayerPerceptron(
            input_size=m1*m2+atom_type_embed_nchanl,
            hidden_size=fitting_hidden_size,
            output_size=1,
            activation_function=activation_function
        )

    def forward(self,
        coords:torch.tensor,
        atom_types:torch.tensor,
        atom_i_types:torch.tensor,
        atom_j_types:torch.tensor,
        atom_i_idxs:torch.tensor,
        atom_j_idxs:torch.tensor,
        shift:torch.tensor
    ):
        coords.requires_grad_(True)
        relative_coords=self.coords_to_relative_coords(
            coords=coords,
            atom_i_idxs=atom_i_idxs,
            atom_j_idxs=atom_j_idxs,
            shift=shift
        )

        generalized_coords=self.relative_coords_to_generalized_coords(
            relative_coords=relative_coords,
            r_cutoff=self.r_cutoff
        )

        atom_type_one_hot=torch.eye(self.max_atom_type+1)
        atom_type_embed_matrix=self.atom_type_embed_net(atom_type_one_hot)
        total_potential_energy=torch.tensor(0.0)

        s_rij=generalized_coords[:,0].reshape(-1,1)
        atom_i_embeded_matrix=atom_type_embed_matrix[atom_i_types]
        atom_j_embeded_matrix=atom_type_embed_matrix[atom_j_types]

        gi1s_before_embed=torch.concat((
            s_rij,
            atom_i_embeded_matrix,
            atom_j_embeded_matrix,
        ),dim=1)

        g_i1s=self.embed_net(gi1s_before_embed).reshape(coords.shape[0],self.sel,self.m1)
        generalized_coords=generalized_coords.reshape(coords.shape[0],self.sel,4)

        left=torch.bmm(
            torch.transpose(gi1s,1,2),
            generalized_coords
        )

        right=torch.bmm(
            torch.transpose(generalized_coords,1,2),
            g_i1s[:,:,:self.m2]
        )

        D_i_s_reshaped=torch.bmm(left,right).reshape(coords.shape[0],self.m1*self.m2)

        feature_vectors=torch.concat((
            D_i_s_reshaped,
            atom_type_embed_matrix[atom_types]
        ),dim=1)

        total_potential_energy=torch.sum(
            self.fitting_net(feature_vectors)
        )

        return total_potential_energy